In [1]:
import pandas as pd
from datetime import timedelta, datetime
import numpy as np
from scipy import stats

import itertools

# data visualization 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Write a function to convert a date to adatetime data type.

def chage_datetime(df):
    df['sale_date'] = df.sale_date.str.replace(' 00:00:00 GMT', '')
    df['sale_date'] = df.sale_date.str[5:]
    df['sale_date'] = df.sale_date.str.replace(' ', '/')
    df['sale_date'] = pd.to_datetime(df['sale_date'], format='%d/%b/%Y')
    return df
    

In [3]:
# Write a function to change a datetime to UTC.

def to_UTC(df):
    df = df.tz_localize('utc')
    return df


In [4]:
# Write a function to parse a date column into 6 additional columns (while keeping the original date): 
# year, quarter, month, day of month, day of week, weekend vs. weekday

def expand_dt(df):
    df['time'] = df.index
    df['year'] = df.time.dt.year
    df['quarter'] = df.time.dt.quarter
    df['month'] = df.time.dt.month
    df['day_of_month'] = df.time.dt.day
    df['day_of_week'] = df.time.dt.dayofweek
    df['weekend'] = ((pd.DatetimeIndex(df.index).dayofweek) // 5 == 1).astype(float)
    df.drop(columns=['time'], inplace=True)
    return df


In [5]:
# Write a function to set the index to be the datetime variable.

def set_dt_index(df):
    df.set_index('sale_date', inplace=True)
    return df


In [6]:
df = prepare.prep_store()
df

,item_id,sale_amount,sale_id,store_id,store_address,store_city,store_state,store_zipcode,item_brand,item_name,item_price,item_upc12,item_upc14,year,quarter,month,day_of_month,day_of_week,weekend
sale_date,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00+00:00,1,13.0,1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,1,1,0.0
2013-01-02 00:00:00+00:00,1,11.0,2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,2,2,0.0
2013-01-03 00:00:00+00:00,1,14.0,3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,3,3,0.0
2013-01-04 00:00:00+00:00,1,13.0,4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,4,4,0.0
2013-01-05 00:00:00+00:00,1,10.0,5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,5,5,1.0
2013-01-06 00:00:00+00:00,1,12.0,6,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,6,6,1.0
2013-01-07 00:00:00+00:00,1,10.0,7,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,7,0,0.0
2013-01-08 00:00:00+00:00,1,9.0,8,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,8,1,0.0
2013-01-09 00:00:00+00:00,1,12.0,9,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,9,2,0.0


In [7]:
# Add a column to your dataframe, sales_total, 
# which is a derived from sale_amount (total items) and item_price.

df['sales_total']  = df.sale_amount * df.item_price

df.head()

,item_id,sale_amount,sale_id,store_id,store_address,store_city,store_state,store_zipcode,item_brand,item_name,item_price,item_upc12,item_upc14,year,quarter,month,day_of_month,day_of_week,weekend,sales_total
sale_date,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00+00:00,1,13.0,1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,1,1,0.0,10.92
2013-01-02 00:00:00+00:00,1,11.0,2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,2,2,0.0,9.24
2013-01-03 00:00:00+00:00,1,14.0,3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,3,3,0.0,11.76
2013-01-04 00:00:00+00:00,1,13.0,4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,4,4,0.0,10.92
2013-01-05 00:00:00+00:00,1,10.0,5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,5,5,1.0,8.40


In [8]:
# Create a new dataframe that aggregates the sales_total and sale_amount(item count)
# using sum and median by day of week.

week_avg = df[['sale_amount','sales_total']].resample('w').mean()

week_avg.head()

,sale_amount,sales_total
sale_date,,
2013-01-06 00:00:00+00:00,30.355667,163.589167
2013-01-13 00:00:00+00:00,29.686286,159.981203
2013-01-20 00:00:00+00:00,29.414286,157.946720
2013-01-27 00:00:00+00:00,29.513714,158.545383
2013-02-03 00:00:00+00:00,31.150571,167.585014


In [9]:
# Explore the pandas DataFrame.diff() function. 
# Create a new column that is the result of the current sales - the previous days sales.

df['date_diff'] = df['sales_total'].diff()
df['date_diff'] = df['date_diff'].fillna(0)

# First line makes a NaN since there is no total the day before, 
# filled it with a 0 since there is no change.

df.head()

,item_id,sale_amount,sale_id,store_id,store_address,store_city,store_state,store_zipcode,item_brand,item_name,item_price,item_upc12,item_upc14,year,quarter,month,day_of_month,day_of_week,weekend,sales_total,date_diff
sale_date,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00+00:00,1,13.0,1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,1,1,0.0,10.92,0.00
2013-01-02 00:00:00+00:00,1,11.0,2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,2,2,0.0,9.24,-1.68
2013-01-03 00:00:00+00:00,1,14.0,3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,3,3,0.0,11.76,2.52
2013-01-04 00:00:00+00:00,1,13.0,4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,4,4,0.0,10.92,-0.84
2013-01-05 00:00:00+00:00,1,10.0,5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,5,5,1.0,8.40,-2.52
